In [ ]:
import pandas as pd
import rasterio
from common import *

use_hvplot()

# Compare SNOBAL SWI to USGS discharge gauge at Almont 

## USGS 

Source: https://nwis.waterdata.usgs.gov/co/nwis/dv/?site_no=09112500&agency_cd=USGS&amp;referred_module=sw

In [ ]:
PD_OPTIONS = dict(
    comment='#',
    parse_dates=True,
    index_col=0,
    names=['Date', 'Discharge'],
    header=0,
    sep='\t',
    dtype={'Discharge': np.float64},
    usecols=[2,3],
)
data = pd.read_csv(
    DATA_DIR / 'USGS-SWI-Almont.csv', 
    **PD_OPTIONS
)

### Convert to daily output

In [ ]:
data.Discharge = data.Discharge * 60 * 60 * 24

data.hvplot.line(x='Date', y='Discharge')

## SNOBAL

In [ ]:
client = start_cluster(5, 16)
client_ip_and_port(client)

In [ ]:
# Convert SNOBAL grid cell from meter to square feet
CELL_AREA = (50.0 * 3.28084)**2
# MM in one foot
FT_IN_MM = 304.8
# Convert to area
CELL_AREA_IN_MM = CELL_AREA / FT_IN_MM

In [ ]:
snobal_em = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy*/erw/*/em.nc').as_posix(),
    data_vars=['SWI', 'evaporation'],
    parallel=True,
)
snobal_em.coords['mask'] = (('y', 'x'), cbrfc_zones())
snobal_em['time'] = snobal_em.time + np.timedelta64(2,'h')

In [ ]:
smrf_precip = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy*/erw/*/precip.nc').as_posix(),
    data_vars=['precip'],
    parallel=True,
)
smrf_precip.coords['mask'] = (('y', 'x'), cbrfc_zones())

In [ ]:
snobal_swe = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy*/erw/*/snow.nc').as_posix(),
    data_vars=['specific_mass'],
    parallel=True,
)
snobal_swe.coords['mask'] = (('y', 'x'), cbrfc_zones())
snobal_swe['time'] = snobal_swe.time + np.timedelta64(2,'h')

### Daily totals for the area

In [ ]:
evaporation = (
    snobal_em.where(snobal_em.mask != 0).evaporation * CELL_AREA_IN_MM
).sum(['x', 'y']).compute()
swi = (
    snobal_em.where(snobal_em.mask != 0).SWI * CELL_AREA_IN_MM
).sum(['x', 'y']).compute()
# Precip input is per hour, add aggregation by day
precip = (
    smrf_precip.where(smrf_precip.mask != 0).precip * CELL_AREA_IN_MM
).resample(time='24H', label='right').sum().sum(['x', 'y']).compute()
swe = (
    snobal_swe.where(snobal_swe.mask != 0).specific_mass * CELL_AREA_IN_MM
).sum(['x', 'y']).compute()

In [ ]:
client.shutdown()

### Calculate added/removed daily SWE 

In [ ]:
swe_diff = swe.diff('time')

### SNOBAL 

In [ ]:
precip.hvplot(label='HRRR Precip', color='limegreen').opts(height=960, width=1280, xlabel='Date', ylabel='Daily Output (mm)') * \
swe_diff.hvplot(label='SNOBAL SWE daily change', color='darkorchid', alpha=0.7) * \
evaporation.hvplot(label='SNOBAL Evaporation', color='chocolate') * \
swi.hvplot(label='SNOBAL SWI', color='tomato', alpha=0.7)*\
hv.HLine(0).opts(color='grey', line_dash='dotted', width=1)

### Moving 7-day average SNOBAL vs. USGS

In [ ]:
swi.rolling(time=7, center=True).mean().hvplot(label='SNOBAL SWI', color='tomato', alpha=0.7).opts(
    title='7-day moving average', xlabel='Date', ylabel='7-Day Mean Output (mm)', height=960, width=1280
) * \
data.rolling(7, center=True).mean().hvplot.line(label='USGS Discharge', color='cadetblue')

## Water year mass balance

In [ ]:
wy2018 = dict(time=slice("2017-10-01", "2018-09-30"))
wy2019 = dict(time=slice("2018-10-01", "2019-09-30"))
wy2020 = dict(time=slice("2019-10-01", "2020-09-30"))
wy2021 = dict(time=slice("2020-10-01", "2021-09-30"))

In [ ]:
def wy_stats(snobal_sel, usgs):
    wy_swe = swe_diff.sel(snobal_sel)
    wy_swi = swi.sel(snobal_sel).sum('time')
    wy_precip = precip.sel(snobal_sel).sum('time')
    
    print(f'Water Year: {usgs.iloc[-1].name.year}')
    print('  HRRR Precip    {:>16,}'.format(float(wy_precip)))
    print('================')
    print('  SWE (in)       {:>16,}'.format(float(wy_swe.where(wy_swe > 0, 0).sum())))
    print('    Ratio Pricip {:>16.2%}'.format(float(wy_swe.where(wy_swe > 0, 0).sum() / wy_precip)))
    print('  SWE (out)      {:>16,}'.format(float(abs(wy_swe.where(wy_swe < 0, 0)).sum())))
    print('================')
    print('  Evaporation    {:>16,}'.format(float(abs(evaporation.sel(snobal_sel).sum('time')))))
    print('================')
    print('  SWI            {:>16,}'.format(float(wy_swi)))
    print('    Ratio SWE in {:>16.2%}'.format(float(wy_swe.where(wy_swe > 0, 0).sum() / wy_swi)))
    print('================')
    print('  USGS Discharge {:>16,}'.format(float(usgs.sum())))
    print('  iSnobal - USGS {:>16,}'.format(float(wy_swi.values - usgs.sum())))
    print('    Ratio:       {:>16.2%}'.format(float(usgs.sum() / wy_swi.values)))

### Results 

### SNOBAL variable definitions
__SWE__: Specific mass per unit area of the snowcover or snow water equivalent

__SWI__: Surface water input is liquid water output from bottom of snowpack or rain on bare ground per unit area

__Evaporation__: Total evaporation and sublimation per unit area from surface of snowpack

In [ ]:
wy_stats(wy2018, data["2017-10-01":"2018-09-30"])

In [ ]:
wy_stats(wy2019, data["2018-10-01":"2019-09-30"])

In [ ]:
wy_stats(wy2020, data["2019-10-01":"2020-09-30"])

In [ ]:
wy_stats(wy2021, data["2020-10-01":"2021-09-30"])